In [1]:
import sys
import os
import numpy as np
sys.path.append(os.path.abspath(os.path.join(r'C:\Users\incognito\OneDrive\Desktop\data_mining')))

from utils.data_utils import (
    test_train_split,
    split_3_channel,
    short_time_fourier_transform,
    integer_label
)

In [2]:
feature_path = "../data/processed/balance_16k/preprocessing/x_train.csv"
label_path = "../data/processed/balance_16k/preprocessing/y_train.csv"

train, test = test_train_split(feature_path, label_path, split=0.1)

s_train_features, train_labels = split_3_channel(train)
train_labels = integer_label(train_labels)

s_test_features, test_labels = split_3_channel(test)
test_labels = integer_label(test_labels)


train_features = short_time_fourier_transform(data=s_train_features)
test_features = short_time_fourier_transform(data=s_test_features)

Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}
Label Mapping: {'Lg': 0, 'P': 1, 'Pg': 2, 'Pn': 3, 'S': 4, 'Sn': 5}


In [3]:
np.save("../data/processed/balance_16k/model_input/train_features.npy", train_features)
np.save("../data/processed/balance_16k/model_input/train_labels.npy", train_labels)

np.save("../data/processed/balance_16k/model_input/test_features.npy", test_features)
np.save("../data/processed/balance_16k/model_input/test_labels.npy", test_labels)

In [4]:
print(f"Train features: {train_features.shape}, Train label: {train_labels.shape} ")
print(f"Test features: {test_features.shape}, Test label: {test_labels.shape} ")

Train features shape: (14672, 3, 50, 50), Test features shape: (1631, 3, 50, 50) 
Train features shape: torch.Size([14672]), Test features shape: torch.Size([1631]) 
